In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import pandas as pd

f = uproot.open("/data/jlai/iris-hep/core_mapped.root")
tree = f['material-tracks']
tree.keys()

['event_id',
 'v_x',
 'v_y',
 'v_z',
 'v_px',
 'v_py',
 'v_pz',
 'v_phi',
 'v_eta',
 't_X0',
 't_L0',
 'mat_x',
 'mat_y',
 'mat_z',
 'mat_r',
 'mat_dx',
 'mat_dy',
 'mat_dz',
 'mat_step_length',
 'mat_X0',
 'mat_L0',
 'mat_A',
 'mat_Z',
 'mat_rho',
 'sur_id',
 'sur_type',
 'sur_x',
 'sur_y',
 'sur_z',
 'sur_r',
 'sur_distance',
 'sur_pathCorrection',
 'sur_range_min',
 'sur_range_max',
 'vol_id']

In [2]:
cols = [
    "v_eta", "mat_step_length", "mat_X0", "sur_range_min", "sur_range_max", "t_X0", "mat_X0", 
    "sur_id", "vol_id", "sur_type", "sur_r", "sur_pathCorrection", "mat_r", "mat_dx", "mat_dz",
    "sur_distance", "sur_type", "sur_range_min", "sur_range_max"
]
arr = tree.arrays(cols, library="ak")

vals = ak.to_numpy(ak.flatten(arr["mat_X0"]))
vals = vals[np.isfinite(vals)]  # drop NaN/Inf 

uniq, counts = np.unique(vals, return_counts=True)
counts_exact = (
    pd.DataFrame({"mat_X0": uniq, "count": counts})
    .sort_values("count", ascending=False)
    .reset_index(drop=True)
)
counts_exact

,mat_X0,count
0,162.002365,958224
1,28.189802,834728
2,93.660835,620837
3,3.929399,480000
4,27.338091,479494
5,17.652525,360000
6,425.382782,352620
7,283.748993,137821
8,280.746033,114472
9,1718.853271,90261


In [4]:
# ---- selection: eta ~ 0 (tune the window as you like) ----
mask = (abs(arr["v_eta"]) < 0.05)
a = arr[mask]

def flat(name): return ak.to_numpy(ak.flatten(a[name], axis=None))

df = pd.DataFrame({
    "sur_id": flat("sur_id"),
    "vol_id": flat("vol_id"),
    "sur_type": flat("sur_type"),
    "sur_r": flat("sur_r"),
    "step_len": flat("mat_step_length"),
    "X0_mm": flat("mat_X0"),
    "pcorr": flat("sur_pathCorrection"),
})

df["pcorr"] = df["pcorr"].replace(0, np.nan)
df["x_over_X0_path"] = df["step_len"] / df["X0_mm"]

# Normal-incidence aggregates
df["step_len_norm"] = df["step_len"] / df["pcorr"]
df["x_over_X0_norm"] = df["step_len_norm"] / df["X0_mm"]

grp = df.groupby(["vol_id", "sur_id", "sur_type"], as_index=False).agg(
    n_steps=("step_len", "size"),
    sur_r_mm=("sur_r", "median"),
    x_path_mm=("step_len", "sum"),
    x_perp_mm=("step_len_norm", "sum"),
    xOverX0_path=("x_over_X0_path", "sum"),
    xOverX0_perp=("x_over_X0_norm", "sum"),
)

grp["X0_eff_mm"] = grp["x_perp_mm"] / grp["xOverX0_perp"]

grp = grp.sort_values(["vol_id", "sur_r_mm", "sur_id"]).reset_index(drop=True)

result = grp[[
    "vol_id", "sur_id", "sur_type", "sur_r_mm",
    "x_perp_mm", "xOverX0_perp", "X0_eff_mm",
    "x_path_mm", "xOverX0_path", "n_steps"
]]

result
# result.to_csv("layer_material_summary_eta0.csv", index=False)


,vol_id,sur_id,sur_type,sur_r_mm,x_perp_mm,xOverX0_perp,X0_eff_mm,x_path_mm,xOverX0_path,n_steps
0,0,144115325514809344,1,24.000000,8.000000e+03,2.267832e+01,352.759766,8.000001e+03,2.267833e+01,10000
1,0,1224979236620599296,1,39.360706,4.289034e+04,1.336534e+02,320.907318,4.289035e+04,1.336534e+02,50764
2,0,1224979374059552768,1,74.805870,4.238108e+04,1.309702e+02,323.593445,4.238109e+04,1.309702e+02,49336
3,0,1224979511498506240,1,120.597031,4.181537e+04,1.306242e+02,320.119690,4.181537e+04,1.306242e+02,48414
4,0,1224979648937459712,1,176.493729,4.212512e+04,1.320383e+02,319.037109,4.212512e+04,1.320383e+02,49022
5,0,1441152018197512192,1,203.000000,2.002700e+04,7.133494e+01,280.746033,2.002700e+04,7.133494e+01,10003
6,0,1729382394617659392,1,247.409409,6.824099e+04,1.689054e+02,404.019012,6.824099e+04,1.689054e+02,49287
7,0,1729382532056612864,1,347.102966,6.855960e+04,1.683091e+02,407.343445,6.855960e+04,1.683091e+02,49522
8,0,1729382669495566336,1,486.880890,6.874928e+04,1.705540e+02,403.093994,6.874928e+04,1.705540e+02,49919
9,0,1729382806934519808,1,646.742920,6.861270e+04,1.685393e+02,407.101990,6.861270e+04,1.685393e+02,49571
